# Optimization of Interaction Parameters in Dilute Binary Hard Spheres #

There is a binary hard sphere mixture of components 1 & 2. It is known that component 1 has a self-interaction parameter of 1, but the self-iteraction parameter of component 2 & the cross-interaction parameter is unknown. Given the RDFs ( rdf_11.dat, rdf_12.dat, & rdf_22.dat ), the self-interaction parameter ( sigma_22 ) and cross-interaction parameter ( sigma_12 ) are found by minimizing the relative entropy. Assume the system is dilute with 25 particles of component 1 & 2 each contained in a cubic box (L = 10) at temperature T = 1.5.

In [1]:
import relentless

Define a directory for the simulation to take place in. This can be an absolute or relative path.

In [2]:
dir = "simulation/"

Define the variables for the simulation. Static variables are defined by setting the variable equal to an integer or float. Design variables are set using <code>relentless.model.DesignVariable(value, name=None, low=None, high=None)</code>

In [3]:
sigma_11 = 1
sigma_12 = relentless.model.variable.DesignVariable(
    value=1,
    name="sigma_12",
    low=0.5,
    high=5.0
)
sigma_22 = relentless.model.variable.DesignVariable(
    value=1,
    name="sigma_22",
    low=0.5,
    high=5.0
)
epsilon = 1

vars = [sigma_12, sigma_22]

Set up the hard core potential. Ensure that the potential is defined in terms of the design variables when applicable. Then attach the potential to the simulation using  <code>relentless.simulate.Potentials(pair_potentials=None, kB=1.0)</code>. The potential's starting point, stopping point, number of subdivisions, and buffer to be added to the end of the stopping point all must be defined. Note: The starting point should be greater than zero. 

In [4]:
wca = relentless.model.potential.LennardJones(types=("1", "2"))
wca.coeff["1", "1"].update({
    "epsilon": epsilon, "sigma": sigma_11, "rmax": 2.**(1./6.)*sigma_11,
    "shift": True
})
wca.coeff["1", "2"].update({
    "epsilon": epsilon, "sigma": sigma_12, "rmax": 2.**(1./6.)*sigma_12,
    "shift": True
})
wca.coeff["2", "2"].update({
    "epsilon": epsilon, "sigma": sigma_22, "rmax": 2.**(1./6.)*sigma_22,
    "shift": True
})

pot = relentless.simulate.Potentials([wca])
pot.pair.start = 1e-2
pot.pair.stop = 5*2**(1/6.)
pot.pair.num = 100
pot.pair.neighbor_buffer = 0.5

Load the RDFs so that they may be used later to define a target ensemble.

In [5]:
rdf_11 = relentless.mpi.world.loadtxt("rdf_11.dat")
rdf_12 = relentless.mpi.world.loadtxt("rdf_12.dat")
rdf_22 = relentless.mpi.world.loadtxt("rdf_22.dat")

Set up a target ensemble and attach the RDFs to each type pair. Ensure that the target ensemble matches the conditions that generated the RDFs. 

In [6]:
target = relentless.model.Ensemble(
    T=1.5,
    V=relentless.model.extent.Cube(L=10.0),
    N={"1": 25, "2": 25}
)
target.rdf["1", "1"] = relentless.model.ensemble.RDF(rdf_11[:, 0], rdf_11[:, 1])
target.rdf["1", "2"] = relentless.model.ensemble.RDF(rdf_12[:, 0], rdf_12[:, 1])
target.rdf["2", "2"] = relentless.model.ensemble.RDF(rdf_12[:, 0], rdf_22[:, 1])

 A dilute simulation assumes $g_{ij}(r) = e^{-\beta u_{ij}(r)}$. The initializer and langevin dynamics do not affect dilute simulations, but they are required. The analyzer implements the physics of the dilute simulation. All three would affect a molecular dynamics simulation. Molecular dynamics simulations can be ran using <code> relentless.simulate.HOOMD(initializer, operations=None) </code> or <code> relentless.simulate.LAMMPS(initializer, operations=None, dimension=3, quiet=True) </code> rather than <code> relentless.simulate.Dilute(initializer, operations=None) </code>.

In [7]:
init = relentless.simulate.InitializeRandomly(
    seed=42,
    T=1.5,
    V=relentless.model.extent.Cube(L=10.0),
    N={"1": 25, "2": 25},
    diameters={"1": 1, "2": 3}
)

anly = relentless.simulate.EnsembleAverage(
    check_thermo_every=1,
    check_rdf_every=1,
    rdf_dr=0.005
)

lgv = relentless.simulate.RunLangevinDynamics(
    steps=5000,
    timestep=0.005,
    T=1,
    friction=0.1,
    seed=2,
    analyzers=[anly]
)

sim = relentless.simulate.Dilute(init, operations=[lgv])

The relative entropy is defined and then optimized as such: 

In [8]:
relent = relentless.optimize.RelativeEntropy(target, sim, pot, anly)

optimizer = relentless.optimize.FixedStepDescent(
    stop=relentless.optimize.GradientTest(0.005, vars),
    max_iter=100,
    step_size=0.1,
    line_search=relentless.optimize.LineSearch(0.1, 3)
)

optimizer.optimize(
    objective=relent,
    design_variables=vars,
    directory=relentless.data.Directory("optimizer")
)

True

Each step of the optimization is logged in the optimization directory specified above. The final step shows that: sigma_12 $\approx 2.00$ and sigma_22 $\approx 4.01$. The RDFs were created with sigma_12 $= 2$ and sigma_22 $= 4$.